## Training a Wide ResNet with Dask Classifier

In [1]:
from dask.distributed import Client

In [2]:
import os
os.chdir('/home/ubuntu/adadamp-experiments')

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import csv
from copy import copy
from adadamp.adadamp import DaskClassifier, DaskClassifierExpiriments

In [3]:
# training client
from dask.distributed import Client

def _prep():
    from distributed.protocol import torch

client = Client(processes=False)
client.run(_prep)
client

Client Scheduler: inproc://172.31.40.124/7299/1 Dashboard: http://172.31.40.124:8787/status,Cluster Workers: 1 Cores: 4 Memory: 16.48 GB


In [4]:
from model import Wide_ResNet

client.upload_file("./exp-dask/model.py")

In [5]:
# load data - https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(root='./exp-dask/data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./exp-dask/data', train=False, download=True, transform=transform)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [6]:
# model for expiriment 1: 
#    “Decaying learning rate” follows the original implementation; 
#     the batch size is constant, while the learning rate repeatedly 
#     decays by a factor of 5 at a sequence of steps
# my understanding is that this is the "control" expeririment, where we are not touching the number of 
# workers nor the batch size
device = "cpu" if not torch.cuda.is_available() else "cuda:0"
model = DaskClassifierExpiriments(
    module=Wide_ResNet,
    module__depth=16,
    module__widen_factor=4,
    module__dropout_rate=0.3,
    module__num_classes=len(classes),
    loss=nn.CrossEntropyLoss,
    optimizer=torch.optim.SGD,
    optimizer__lr=0.1,
    optimizer__momentum=0.9,
    optimizer__nesterov=True,
    optimizer__weight_decay=0.5e-3,
    batch_size=128,
    max_epochs=200,
    device=device,
    grads_per_worker=128,
    client=client
)

In [7]:
def write_stats(hist, epoch, exp):
    toCSV = hist
    with open('./exp-dask/stats/{}/exp-v0-ep{}.csv'.format(exp, epoch), 'w', encoding='utf8', newline='') as output_file:
        fc = csv.DictWriter(output_file, fieldnames=toCSV[0].keys())
        fc.writeheader()
        fc.writerows(toCSV)

In [8]:
def train(model, train_set, test_set, n_epochs=200, epoch_sched=[], lr_sched=[], bs_sched=[], log_interval=1, exp='increasing-bs'):
    """
    Train based on expiriment params
    
    Parameters:
    epoch_sched: update lr and bs at epochs in this list
    lr_sched: update lr to value at matching epoch. Should be same length as epoch_sched
    bs_sched: update bs to value at matching epoch. Should be same length as epoch_sched
    """
    assert len(epoch_sched) == len(lr_sched) == len(bs_sched), "Invalid schedules. Epoch, lr and bs schedules should all be the same length."
    
    epochs = copy(epoch_sched)
    lrs = copy(lr_sched)
    bss = copy(bs_sched)
    
    history = []
    for epoch in range(n_epochs):
        # check for updates
        if len(epochs) > 0 and epochs[0] == epoch:
            lr = lrs.pop(0)
            bs = bss.pop(0)
            epochs.pop(0)
            model.set_lr(lr)
            model.set_bs(bs)
            print("[UPDATE] Updated model params:\n\tlr: {}\n\tbs: {}".format(lr, bs))
        # run
        print("[Epoch {}]".format(epoch), end="")
        model.partial_fit(train_set)
        score = model.score(test_set)
        datum = {"epoch": epoch, "score": score, **model.get_params(), **model.meta_}
        print(" Score: {}".format(score))
        history.append(datum)
        
        if epoch % log_interval == 0:
            write_stats(history, epoch, exp)
    return history

In [9]:
# ie "Update LR to 0.1 and bs t0 640 on 60th epoch"
exp1_epochs = [0, 60, 120, 180]
exp1_lr = [0.1, 0.1, 0.1, 0.1]
exp1_bs = [128, 640, 3200, 16000]
# train
hist = train(model, 
             train_set, 
             test_set, 
             n_epochs=200, 
             epoch_sched=exp1_epochs, 
             lr_sched=exp1_lr, 
             bs_sched=exp1_bs, 
             log_interval=20,
             exp='increasing-bs'
            )

[UPDATE] Updated model params:
	lr: 0.1
	bs: 128
[Epoch 0] Score: 0.2782999873161316
[Epoch 1] Score: 0.2807999849319458
[Epoch 2] Score: 0.27410000562667847
[Epoch 3] Score: 0.27730000019073486
[Epoch 4] Score: 0.2777999937534332
[Epoch 5] Score: 0.2757999897003174
[Epoch 6] Score: 0.2800999879837036
[Epoch 7] Score: 0.27950000762939453
[Epoch 8] Score: 0.2800999879837036
[Epoch 9] Score: 0.243599995970726
[Epoch 10] Score: 0.23749999701976776
[Epoch 11] Score: 0.2433999925851822
[Epoch 12] Score: 0.24210000038146973
[Epoch 13] Score: 0.2458999902009964
[Epoch 14] Score: 0.25759997963905334
[Epoch 15] Score: 0.2630999982357025
[Epoch 16] Score: 0.25949999690055847
[Epoch 17] Score: 0.26729997992515564
[Epoch 18] Score: 0.2694000005722046
[Epoch 19] Score: 0.2678999900817871
[Epoch 20] Score: 0.2775999903678894
[Epoch 21] Score: 0.28200000524520874
[Epoch 22] Score: 0.26969999074935913
[Epoch 23] Score: 0.26639997959136963
[Epoch 24] Score: 0.2727999985218048
[Epoch 25] Score: 0.275000

In [10]:
write_stats(hist, '-final', 'increasing-bs')

In [ ]:
# ie "Update LR to 0.1 and bs t0 640 on 60th epoch"
exp2_epochs = [0, 60, 120, 180]
exp2_lr = [0.1, 
           0.1, 
           0.1 / 5, 
           0.1 / 5 / 5
          ]
exp2_bs = [128, 640, 640, 640]
# train
hist = train(model, train_set, test_set, n_epochs=200, epoch_sched=exp1_epochs, lr_sched=exp1_lr, bs_sched=exp1_bs)

In [ ]:
# ie "Update LR to 0.1 and bs t0 640 on 60th epoch"
exp3_epochs = [0, 60, 120, 180]
exp3_lr = [0.1, 
           0.1 / 5, 
           0.1 / 5 / 5,
           0.1 / 5 / 5 / 5
          ]
exp3_bs = [128, 128, 128, 128]
# train
hist = train(model, train_set, test_set, n_epochs=200, epoch_sched=exp1_epochs, lr_sched=exp1_lr, bs_sched=exp1_bs)

In [ ]:
toCSV = model.curr_metas
with open('./exp-dask/exp1-v0.csv', 'w', encoding='utf8', newline='') as output_file:
    fc = csv.DictWriter(output_file, fieldnames=toCSV[0].keys())
    fc.writeheader()
    fc.writerows(toCSV)